In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [4]:
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hand_tips = [8,12]
news = False
xp,yp=0,0
mode="line"
paint = np.ones((480, 640))*255
paint = paint.astype('uint8')
paint = np.zeros((480,640,3),np.uint8)

In [6]:
cap = cv2.VideoCapture(0)
with mp_hand.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5,max_num_hands=1) as hands:
    while True:
        istrue,img = cap.read()#True or img
        cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        result = hands.process(img)
        cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        if result.multi_hand_landmarks:
            data = []
            hand_num = []
            
            for index,hand_land in enumerate(result.multi_hand_landmarks):
                for keynum,hand_key in enumerate(result.multi_hand_landmarks[index].landmark):
                    h,w,f = img.shape
                    coorx,coory =int(hand_key.x*w),int(hand_key.y*h)
                    data.append([keynum,coorx,coory])
                    mp_drawing.draw_landmarks(img,hand_land,mp_hand.HAND_CONNECTIONS)
            ix,iy=data[8][1],data[8][2]
    
            for oc in hand_tips:
                if data[oc][2]>data[oc-2][2]:
                    hand_num.append(0)
                else:
                    hand_num.append(1)
            
            if hand_num[0] and hand_num[1]==0:
                if ix < 400 and iy < 50 and ix > 150:
                    if ix < 200:
                        mode="line"
                    elif ix<250:
                        mode="rectangle"
                    elif ix < 300:
                        mode="draw"
                    elif ix<350:
                        mode="circle"
                    elif ix<400:
                        mode="erase"
                
                if mode=="draw":
                    xp,yp=0,0
                    pass
                elif mode=="line":
                    if news:
                        cv2.line(paint, (xs, ys), (data[8][1],data[8][2]), (255,0,255), 4,cv2.LINE_AA)
                        news = False
                elif mode=="rectangle":
                    if news:
                        cv2.rectangle(paint, (xs, ys), (data[8][1],data[8][2]), (255,0,255), 4)
                        news = False
                elif mode=="circle":
                    if news:
                        cv2.circle(paint, (xs, ys), int(((xs-ix)**2 + (ys-iy)**2)**0.5), (255,0,255), 4)
                        news = False

            
            if hand_num[0] and hand_num[1]:
                if mode=="draw":
                    if xp==0:
                        xp,yp=data[8][1],data[8][2]
                    cv2.line(paint,(data[8][1],data[8][2]),(xp,yp),(255,0,255),2,cv2.LINE_AA)
                    xp,yp=data[8][1],data[8][2]
                elif mode=="line":
                    if not(news):
                        xs,ys=data[8][1],data[8][2]
                        news = True
                    cv2.line(img, (xs, ys), (data[8][1],data[8][2]), (255,0,255), 4)  
                elif mode=="rectangle":
                    if not(news):
                        xs,ys=data[8][1],data[8][2]
                        news = True
                    cv2.rectangle(img, (xs, ys), (data[8][1],data[8][2]), (255,0,255), 4)  
                   
                    
                elif mode=="circle":
                    if not(news):
                        xs,ys=data[8][1],data[8][2]
                        news = True
                    cv2.circle(img, (xs, ys), int(((xs-ix)**2 + (ys-iy)**2)**0.5), (255,255,0), 4)
                    
                elif mode=="erase":
                    cv2.circle(img, (ix, iy), 30, (0,0,0), -1)
                    cv2.circle(paint, (ix, iy), 30, (0,0,0), -1)
                 
                
                
                    


                
           
          
       # main = cv2.addWeighted(img, 0.5, paint, 0.5, 0.0)  
        cv2.flip(img,1)
        imgg=cv2.cvtColor(paint,cv2.COLOR_BGR2GRAY)
        n,imggv = cv2.threshold(imgg,50,255,cv2.THRESH_BINARY_INV)
        imggv = cv2.cvtColor(imggv,cv2.COLOR_GRAY2BGR)
        img = cv2.bitwise_and(img,imggv)
        img = cv2.bitwise_or(img,paint)
        #kl = cv2.addWeighted()
        ml = 150
        max_x, max_y = 250+ml, 50
        tools = cv2.imread("tools.png")
        img[0:50, 150:400] = tools
        cv2.imshow("window",img)
        cv2.imshow("paint",paint)
        #cv2.imshow("Ai painter",main)
                  
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()   

KeyboardInterrupt: 

: 